# 0. 구글 드라이브와 연결 및 각종 라이브러리 설정

### 구글 드라이브와 연결

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 라이브러리 설치

In [3]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 12.5 MB/s 
     |████████████████████████████████| 453 kB 51.9 MB/s 


In [5]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 5.3 MB/s 


### 라이브러리 불러오기

In [68]:
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
import re
from soynlp.normalizer import *
from konlpy.tag import Okt

In [7]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Unzipping grammars/book_grammars.zip.
       | Downloading 


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

# 1. 데이터 로드

### 데이터 로드

In [8]:
train0 = pd.read_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data set/감성대화말뭉치(최종데이터)_Training.xlsx', engine='openpyxl')

In [9]:
validation0 = pd.read_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data set/감성대화말뭉치(최종데이터)_Validation.xlsx', engine='openpyxl')

In [10]:
nsmc0 = pd.read_csv('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data set/nsmc_data.txt', sep='\t')

### 가져온 데이터 확인

In [11]:
train0.head(2)

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,3926,노년,남성,"건강, 죽음",만성질환 유,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN


In [12]:
validation0.head(2)

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,23093,청년,여성,"진로, 취업, 직장",해당없음,분노,툴툴대는,요즘 부모님과 많이 부딪혀.,어떤 일로 부딪히시나요?,난 하고 싶은 일이 있는데 부모님은 안정적인 일만 하라고 하셔. 내 마음대로 하지도...,부모님과의 충돌로 힘드시겠군요.,응. 그래도 난 내가 하고 싶은 일을 위해 노력할 거야.,자신이 하고 싶은 일을 목표로 하기로 하셨군요.,NaN,NaN
1,32848,청소년,남성,가족관계,해당없음,슬픔,비통한,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.,어머니께서 집을 나가셨다니 정말 슬프셨겠어요. 좀 더 자세히 얘기해주시겠어요?,엄마도 엄마만의 인생이 있을 테니 가지 말라고 할 수가 없었어. 그래서 더 마음이 ...,정말 많이 답답하셨겠어요. 어떻게 하는 게 기분이 좀 풀릴까요?,일단 좀 자야겠어.,일단 휴식을 가질 생각이시군요.,NaN,NaN


In [13]:
nsmc0.head(2)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1


### sample의 개수 확인

In [14]:
print('train0의 sample 수: {}개'.format(train0.shape[0]))
print('validation0의 sample 수: {}개'.format(validation0.shape[0]))
print('nsmc0 sample 수: {}개'.format(nsmc0.shape[0]))
print('모든 샘플의 수: {}개'.format(train0.shape[0]+validation0.shape[0]+nsmc0.shape[0]))

train0의 sample 수: 40879개
validation0의 sample 수: 5130개
nsmc0 sample 수: 200000개
모든 샘플의 수: 246009개


# 2. 데이터 정제
### 중복된 값, 결측 값

### nsmc 데이터 확인 및 정제

`결측값 제거`

In [48]:
# 결측값을 확인

nsmc0.isnull().sum()

id          0
document    8
label       0
dtype: int64

In [16]:
nsmc0.loc[nsmc0['document'].isnull()]

,id,document,label
46471,6369843,NaN,1
60735,511097,NaN,1
77665,2172111,NaN,1
84098,402110,NaN,1
127017,5942978,NaN,0
172375,5026896,NaN,0
173526,1034280,NaN,0
197279,1034283,NaN,0


In [17]:
nsmc1 = nsmc0.dropna(how='any')

In [18]:
# 결측값을 확인

nsmc1.isnull().sum()

id          0
document    0
label       0
dtype: int64

`중복값 제거`

In [19]:
print('nsmc1 데이터 샘플의 수: {}'.format(nsmc1.shape[0]))
print('nsmc1 데이터의 document 종류의 수: {}'.format(nsmc1['document'].nunique()))

nsmc1 데이터 샘플의 수: 199992
nsmc1 데이터의 document 종류의 수: 194543


In [20]:
nsmc2 = nsmc1.drop_duplicates(subset=['document'], ignore_index=True)

In [21]:
print('nsmc2 데이터 샘플의 수: {}'.format(nsmc2.shape[0]))
print('nsmc2 데이터의 document 종류의 수: {}'.format(nsmc2['document'].nunique()))

nsmc2 데이터 샘플의 수: 194543
nsmc2 데이터의 document 종류의 수: 194543


`노이즈 제거`

In [24]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㄷ' in sent)]

,id,document,label
45,10142929,내가 남자라그런가 이거 겁나 긴장감있고 흥미진진하던데..ㄷㄷ 나만그런가워낙에 격투씬...,1
183,8484420,와 쩌네..ㄷㄷㄷ 캬..ㅎㅎ,1
224,8851783,내가무서운걸잘못보는데진짜무서워하면서도재미있어서잘봤음.박한별송지효조안다이쁘더라..조안...,1
353,8828360,과하지않다 강간범새끼들은 저것보다 더해줘도 분하다 강간은 정신을 완전히 난도질하는거...,1
513,10125993,"할머니 돌아가시고 우는데, 우는소리가 낯익다...네이티리ㅡ그녀였구나!!!어쩐지. 연...",1
...,...,...,...
193212,9045400,이감독의 다른영화도과연 스토리가 산인지 보고싶을정도로 소름끼친ㄷㅏ;; 2008년에 ...,0
193227,7497120,와 ㅋㅋ 대박 ㅋㅋㅋㅋ정말 앵간한 영화 다보는편인데 이거 정말주성치만 믿엇다 .. ...,0
193282,7865185,1점도 아깝ㄷ,0
194397,5501124,액션도 너무 가짜로 치는거 티가나고 ..무엇보다 배우들의 발 연기가 ㄷㄷ,0


In [31]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㅗ' in sent)]['label'].value_counts()

0    75
1    16
Name: label, dtype: int64

In [32]:
p = re.compile('[ㅎ]{2,}')

In [33]:
kind = []
for i in range(nsmc2.shape[0]):
   kind.extend(p.findall(nsmc2['document'][i]))

for c in list(set(kind)):
  print('{}의 수: {}'.format(c, kind.count(c)))

ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ의 수: 2
ㅎㅎㅎㅎㅎㅎㅎ의 수: 3
ㅎㅎㅎㅎㅎㅎㅎㅎ의 수: 7
ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ의 수: 3
ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ의 수: 1
ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ의 수: 1
ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ의 수: 2
ㅎㅎㅎ의 수: 460
ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ의 수: 1
ㅎㅎㅎㅎ의 수: 120
ㅎㅎㅎㅎㅎ의 수: 19
ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ의 수: 2
ㅎㅎㅎㅎㅎㅎ의 수: 8
ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ의 수: 1
ㅎㅎ의 수: 2049


In [34]:
nsmc2

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
194539,3302770,쓰.레.기,0
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [36]:
nsmc3 = nsmc2.copy()
nsmc3

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
194539,3302770,쓰.레.기,0
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [37]:
nsmc3['document'] = nsmc3['document'].map(lambda sent: re.sub('ㅜ', 'ㅠ', sent))

In [38]:
nsmc3    # 'ㅜ'를 'ㅠ'로 바꾼 data

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
194539,3302770,쓰.레.기,0
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [49]:
nsmc4 = nsmc3.copy()

In [52]:
nsmc4['document'] = nsmc4['document'].map(lambda sent: re.sub('[^ㅋㅎㅡㅉㅠ 가-힣]', '', sent))

In [53]:
nsmc4    # 영어와 특수문자는 지워준 data

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ,0
194539,3302770,쓰레기,0
194540,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [54]:
nsmc5 = nsmc4.copy()

In [55]:
nsmc5

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ,0
194539,3302770,쓰레기,0
194540,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [59]:
nsmc5['document'] = nsmc5['document'].map(lambda sent: emoticon_normalize(sent, num_repeats=2))

In [62]:
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅋㅋㅋ','ㅋㅋ',sent))
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅎㅎㅎ','ㅎㅎ',sent))
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅠㅠㅠ','ㅠㅠ',sent))
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅡㅡㅡ','ㅡㅡ',sent))
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅉㅉㅉ','ㅉㅉ',sent))

In [78]:
nsmc6 = nsmc5.copy()


In [79]:
nsmc6['document'] = nsmc6['document'].map(lambda sent: re.sub('^ +', '', ))

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ,0
194539,3302770,쓰레기,0
194540,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [81]:
nsmc6['document'].replace('', np.nan, inplace=True)

In [84]:
nsmc6

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ,0
194539,3302770,쓰레기,0
194540,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [88]:
nsmc3.loc[nsmc6['document'].isnull()]

,id,document,label
202,10216462,goodgoodgoodgoodgoodgoodgood,1
607,8085167,Very interesting,1
660,3579401,Killing time yong good,1
982,5338464,GOOD~,1
1197,7887700,♥♥,1
...,...,...,...
193851,4852134,.............................,0
193876,9233162,Real Trash Movie,0
193899,4602734,2,0
194144,2259844,ㄲ ㅓ ㅈ ㅕ,0


In [83]:
print(nsmc5['document'].isnull().sum())
print(nsmc6['document'].isnull().sum())

# 주의!!!! null 값이 Nan으로 되어있지 않을 수 있다!!

0
1108


In [89]:
nsmc7 = nsmc6.copy()

In [91]:
nsmc7 = nsmc7.dropna(how='any')

In [93]:
print(nsmc7['document'].isnull().sum())

# 결측값이 모두 사라졌다.

0


In [94]:
nsmc7

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ,0
194539,3302770,쓰레기,0
194540,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [104]:
nsmc8 = nsmc7.copy()
nsmc8['document'].nunique()

# 겹치는 것이 존재한다.

190368

In [105]:
nsmc8 = nsmc8.drop_duplicates(subset=['document', 'label'], ignore_index=True)

In [109]:
nsmc8['document'].shape[0], nsmc8['document'].nunique()

# document는 같지만 label이 다른 것이 존재하는 듯..(버리지 않는 것이 좋으려나?)

(190530, 190368)

In [110]:
nsmc8

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
190525,4834376,개연성이 없어요 별루다,0
190526,8963373,포켓 몬스터 짜가 ㅡㅡ,0
190527,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
190528,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [111]:
okt.morphs('포켓 몬스터 짜가 ㅡㅡ')

['포켓', '몬스터', '짜가', 'ㅡㅡ']

In [47]:
p = re.finditer('ㅋ{1,}', 'ㅋㅋ 안녕ㅋ')
for i in p:
  print(i.span())

(0, 2)
(5, 6)


In [77]:
re.sub('^ +', '', ' ')

''

In [76]:
okt.morphs('   아   ㅋㅋㅋ')

['아', 'ㅋㅋㅋ']